# Experimentación sobre la Evaluación de Algoritmos


En este notebook recogeremos las partes clave sobre la experimentación realizada en esta práctica, tanto en código como en resultados.

Empezaremos por la presentación de los 3 algoritmos usados: 2 DE's (el nuestro y el de la biblioteca Scipy) y un SADE de la biblioteca Pyade:

## Clase EA de nuestra implementación

In [ ]:
class EA(object):

    def __init__(self, minfun, bounds, psize):
        self.minfun = minfun
        self.bounds = bounds
        self.psize = psize
        
    def best(self, population):
        population.ascendant_sort()
        best = population.get_population()[0]
        return best

    def evaluate(self, genome):
        genome.set_score(self.minfun(genome.get_genes()))

    def run(self, iteraciones):
        # Inicializacion de la poblacion
        pop = Population(self.psize, self.bounds)
        for i in pop.get_population():
            self.evaluate(i)
        contador = 0
        while contador < iteraciones:
            target_vector = self.best(pop)
            mutant_vector = MOperator(pop.get_population(), bounds).apply(target_vector)
            self.evaluate(mutant_vector)
            trial_vector = COperator(pop.get_population(), bounds).apply(mutant_vector, target_vector, 0.5)
            self.evaluate(trial_vector)
            pop = ROperator(pop.get_population()).apply(target_vector, trial_vector, pop)
            contador += 1
        winner = self.best(pop)
        return winner

## DE de la biblioteca Scipy

In [ ]:
from scipy.optimize import differential_evolution


best_scipy = differential_evolution(func, bounds, strategy='best2bin',
                                    popsize=25, maxiter=100, recombination=0.5)


## SADE de la biblioteca Pyade

In [ ]:
import pyade.sade as pyade

##Recogemos el índice 1 del valor devuelto por la función apply ya que es 
##el fitness que retorna la ejecución del algoritmo.

best_pyade = pyade.apply(population_size=25, individual_size=10,
                         bounds=numpy.ndarray((10, 2)),
                        func=func, opts=None, callback=None,
                         max_evals=100, seed=None)[1]




## Modo de ejecución

In [ ]:
total_medias_de = []
total_medias_scipy = []
total_medias_sade = []

for func in benchmarks:
    myEA = EA(func, bounds, 25)
    lista_de = []
    lista_scipy = []
    lista_sade = []
    for i in range(0, 10):
        best = myEA.run(100)
        best_scipy = differential_evolution(func, bounds, strategy='best2bin',
                                            popsize=25, maxiter=100, recombination=0.5)
        best_pyade = pyade.apply(population_size=25, individual_size=10, bounds=numpy.ndarray((10, 2)),
                                 func=func, opts=None, callback=None, max_evals=100, seed=None)[1]
        lista_de.append(best.get_score())
        lista_scipy.append(best_scipy.fun)
        lista_sade.append(best_pyade)
    

Como podemos observar el modo de ejecución es bastante sencillo: tenemos un 
bucle que recorre las funciones de benchmarking y otro bucle anidado dentro 
del primero en el que se ejecuta 10 veces cada algoritmo sobre la misma función

## Cálculo de estadísticas y formato de presentación

In [ ]:
print("Fitness de DE")
    for i in lista_de:
        print(i)
    print("\n")

    print("Estadísticas de DE")
    media_de = statistics.mean(lista_de)
    total_medias_de.append(media_de)
    print(str(media_de))
    print(str(statistics.stdev(lista_de)))
    print(str(statistics.median(lista_de)))
    print(str(statistics.quantiles(lista_de, n=len(lista_de) + 1)[0]))
    print(str(statistics.quantiles(lista_de, n=len(lista_de) + 1)[len(lista_de) - 1]) + "\n")
    
    res = kruskal(lista_de, lista_scipy, lista_sade)
    print("Resultados de Kruskal en la función " + str(func.__name__) + ": " + str(res))

res = friedmanchisquare(total_medias_de, total_medias_scipy, total_medias_sade)
print("Resultados de Friedman: " + str(res))
    

Para el cálculo de las medidas estadísticas hemos optado por usar la biblioteca statistics de python, la cual no contaba con las funciones para obtener los valores mínimos y máximos de ca

El formato de impresión de resultados es el mismo para los otros dos algoritmos,solo hay que cambiar las estructuras de datos donde se almacenan los resultados obtenidos de la ejecución de estos.

Hemos optado por un formato simplista a la hora de especificar que es cada dato ya que estos se iban a especificar en las tablas de la memoria, ademas de para facilitarnos el traspaso de esos datos a dichas tablas.

In [ ]:
Fitness de DE
3.7511972338327175
0.6747256113582627
0.24265402641538952
8.814826221348667
1.328444197097738
0.000812309596253574
0.41475018503875244
3.0897599271968663
0.621826416826631
0.7453245340639049


Estadísticas de DE
1.9684320662775183
2.707296830449243
0.7100250727110837
0.000812309596253574
8.814826221348667

Fitness de Scipy
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


Estadísticas de Scipy
0.0
0.0
0.0
0.0
0.0

Fitness de Pyade
0.00242593487413689
0.0031722994641388154
0.0024976900923057285
0.004352575439392395
0.001923501980190291
0.001402370153307329
0.003861735998482732
0.005413986253516438
0.0035321570738812316
0.004104829153986344


Estadísticas de Pyade
0.0032687080483338195
0.0012249262822273226
0.0033522282690100235
0.001402370153307329
0.005413986253516438
Resultados de Kruskal en la función sphere: KruskalResult(statistic=24.378752886836025, pvalue=5.084181813288585e-06)
Fitness de DE
4.440892098500626e-16
8.425116087476866
0.9459286561463398
4.440892098500626e-16
1.5969879387359183
1.1790169272030693
3.9098638329822575
3.847899178708421
0.685792417032197
1.6462816573712442


Estadísticas de DE
2.2236886695656315
2.574313680406608
1.3880024329694938
4.440892098500626e-16
8.425116087476866

Fitness de Scipy
4.440892098500626e-16
2.0133152362567164
4.440892098500626e-16
4.440892098500626e-16
4.440892098500626e-16
1.1551485027098392
4.440892098500626e-16
4.440892098500626e-16
4.440892098500626e-16
4.440892098500626e-16


Estadísticas de Scipy
0.31684637389665593
0.6979247060107421
4.440892098500626e-16
4.440892098500626e-16
2.0133152362567164

Fitness de Pyade
0.14034689190248928
0.08297121231751037
0.05883931180723012
0.12411213677860689
0.07747204599951418
0.07384239467454679
0.07691115961356987
0.06590403385561361
0.17211387560244562
0.058364571373136886


Estadísticas de Pyade
0.09308776339246636
0.038782781111010314
0.07719160280654203
0.058364571373136886
0.17211387560244562
Resultados de Kruskal en la función ackley: KruskalResult(statistic=9.968498845265595, pvalue=0.00684491373717667)
Fitness de DE
0.0
2098.8385671510773
3137.9553371377838
590.8008956417589
0.0
563.6313022480872
397.0828220876558
1106.040267290261
1891.6494039989507
879.5983869308797


Estadísticas de DE
1066.5596982486454
1015.2256291704297
735.1996412863193
0.0
3137.9553371377838

Fitness de Scipy
0.0
1.855302906961063e-12
1.0309396407475561e-10
4.4504316133711224e-11
4.238214202464481e-11
0.0
8.112318774587342e-11
3.4725192045557266e-11
5.442159495825945e-13
4.283807068660394e-11


Estadísticas de Scipy
3.5106639156768996e-11
3.606496235901961e-11
3.855366703510104e-11
0.0
1.0309396407475561e-10

Fitness de Pyade
10.301449701221445
15.446182617920314
1.1276176672901217
9.503977666543145
0.2626367514440256
27.869594204211158
3.1575562063765417
15.708681609215533
2.121024516710942
25.33275131465956


Estadísticas de Pyade
11.083147225559278
9.931449176485712
9.902713683882295
0.2626367514440256
27.869594204211158
Resultados de Kruskal en la función rosenbrock: KruskalResult(statistic=14.18381270903009, pvalue=0.000831810128175261)
Fitness de DE
20.32317317171118
16.442137000583955
17.504830867070815
14.044318824852677
35.442073202252416
17.694989996875663
9.976246108176861
2.526263150985921
0.23675212249573008
16.23446083163532


Estadísticas de DE
15.042524527664053
9.792493020688815
16.338298916109636
0.23675212249573008
35.442073202252416

Fitness de Scipy
12.934462704394257
21.889058908062694
19.899145831695
11.939498609481534
28.853701459037655
18.904201933496257
8.954626476159767
11.939498609481378
20.894120047689096
15.919329800040812


Estadísticas de Scipy
17.212764437953844
6.007379499847858
17.411765866768533
8.954626476159767
28.853701459037655

Fitness de Pyade
15.531581426878988
19.949258941467306
19.79711636471839
18.49992852610082
20.518963254356862
7.714984978826578
6.753942021699379
12.903070151202977
17.626901818879077
13.340564522390434


Estadísticas de Pyade
15.263631200652082
4.9991541439227705
16.57924162287903
6.753942021699379
20.518963254356862
Resultados de Kruskal en la función rastrigin: KruskalResult(statistic=0.5909677419354864, pvalue=0.7441714173911155)
Fitness de DE
1.0
1.00021894788018
1.0000646517494796
1.0
1.0002280712536988
1.0
1.001916088852224
1.0000552649845997
1.001281171864636
1.0003968692968699


Estadísticas de DE
1.0004161065881687
0.0006537111905801868
1.0001417998148296
1.0
1.001916088852224

Fitness de Scipy
1.0000000000015006
1.0
1.0
1.0
1.0000000000000258
1.0000000000000062
1.0000000000000178
1.0000000000004197
1.0000000000003084
1.0


Estadísticas de Scipy
1.0000000000002278
4.719793740640453e-13
1.000000000000012
1.0
1.0000000000015006

Fitness de Pyade
1.0000208515442737
1.0000296468701984
1.00004017738462
1.0000133668430053
1.0000187258359332
1.0000325839032633
1.000064324009855
1.0000416866753854
1.000053404240749
1.000030726381874


Estadísticas de Pyade
1.0000345493689158
1.582500805006946e-05
1.0000316551425685
1.0000133668430053
1.000064324009855
Resultados de Kruskal en la función griewank: KruskalResult(statistic=11.448434332056776, pvalue=0.0032659089399817397)
Fitness de DE
0.0
1.0841182707202182
1.9753947610574754
1.9479692010147027
1.8355479161106507
2.3868664887400746
1.1505953343332669
2.4496860011347152
0.4641165969902521
3.4107443792014145


Estadísticas de DE
1.670503894930277
1.0113812675995564
1.8917585585626768
0.0
3.410744379201414

Fitness de Scipy
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


Estadísticas de Scipy
0.0
0.0
0.0
0.0
0.0

Fitness de Pyade
0.0483094655102336
0.03744133025322616
0.05575811493683536
0.056071185306722254
0.03877368543504956
0.05680902006944151
0.05891281316728579
0.0622690874397138
0.050280852400073586
0.057802190025164964


Estadísticas de Pyade
0.052242774454374655
0.008456474076292119
0.055914650121778806
0.03744133025322616
0.0622690874397138
Resultados de Kruskal en la función schwefel_2_21: KruskalResult(statistic=22.182456140350876, pvalue=1.524547118981126e-05)
Fitness de DE
0.0
0.0
2.522663494673343
1.2210507001724045
2.733207004424902
0.0
4.180698423399851
5.671641967515823
1.9910792013515124
6.3922764273247195


Estadísticas de DE
2.4712617218862554
2.3267819382303485
2.2568713480124276
0.0
6.3922764273247195

Fitness de Scipy
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


Estadísticas de Scipy
0.0
0.0
0.0
0.0
0.0

Fitness de Pyade
0.18172997994943899
0.17833965030761742
0.1998838796837818
0.13526722972097305
0.1495540963988055
0.10355907348238466
0.21506666133979485
0.21353854627802585
0.2061121955020195
0.17948466208605368


Estadísticas de Pyade
0.17625359747488953
0.036619013162668655
0.18060732101774635
0.10355907348238466
0.21506666133979485
Resultados de Kruskal en la función schwefel_2_22: KruskalResult(statistic=15.654805131929317, pvalue=0.00039865962873010665)
Fitness de DE
0.0
1.2159393239737075
81.61993270666179
0.07548852286543478
166.18704832556082
0.7499681933612251
8.972776666439817
69.5020030283278
85.49892461714592
2.707220614638909


Estadísticas de DE
41.65293019989754
56.92174302718393
5.839998640539363
0.0
166.18704832556082

Fitness de Scipy
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


Estadísticas de Scipy
0.0
0.0
0.0
0.0
0.0

Fitness de Pyade
0.03317674863535275
0.08711055893066344
0.052742076139519464
0.014674303256267942
0.02171212138091982
0.04870840609294834
0.07585459311138136
0.06569378760965448
0.04071347938389759
0.01369613850052744


Estadísticas de Pyade
0.04540822130411326
0.025441942530868546
0.044710942738422965
0.013696138500527442
0.08711055893066344
Resultados de Kruskal en la función schwefel_1_2: KruskalResult(statistic=21.786292397660805, pvalue=1.8585177587672292e-05)
Fitness de DE
10.360487348302945
6.968209854089093
18.488031499176973
6.541513836790551
11.216183273709767
2.8643100980509026
3.420124965615564
21.063404159426074
8.979560340201186
7.135370499315873


Estadísticas de DE
9.703719587467893
5.960890365365734
8.057465419758529
2.8643100980509026
21.063404159426074

Fitness de Scipy
6.894336114962439
6.652864133265149
9.08081617145243
8.760645519469872
7.921293447914139
7.608818055870414
8.691321134202626
5.747895548210275
8.178585643418337
6.318220684297204


Estadísticas de Scipy
7.585479645306289
1.1370103846984934
7.765055751892277
5.747895548210275
9.08081617145243

Fitness de Pyade
2.7280760263221246
2.5363290454955814
2.4767535049427116
3.5731923957990888
2.633889371276125
2.404112857816869
3.2629668380292634
3.1017603926521513
3.4055404154914237
3.2805822717840045


Estadísticas de Pyade
2.9403203119609342
0.4304348990646381
2.914918209487138
2.404112857816869
3.5731923957990888
Resultados de Kruskal en la función extended_f_10: KruskalResult(statistic=17.194838709677427, pvalue=0.00018458151896825312)
Fitness de DE
1.2712461327022804
0.0
20.67016669462263
9.883190814300495
3.8278866431726635
1.0882672003702607
2.6156200819362847
25.50282702958363
1.2960152425544065
0.06427341863029154


Estadísticas de DE
6.621949325787294
9.206300215974503
1.9558176622453456
0.0
25.502827029583628

Fitness de Scipy
3.6192345761242324
4.0321614064041835
5.2630611659776125
0.0
4.669018629858509
1.4627108840100955
0.41292683027581717
2.569450522389959
0.0
2.5694505223901047


Estadísticas de Scipy
2.4598014537430513
1.9397790275101991
2.569450522390032
0.0
5.2630611659776125

Fitness de Pyade
0.21436679451702587
0.2118774478705867
0.017962089859151518
2.6383142469779868
0.16568100484639597
0.08771768658961085
0.1264396967134566
0.25359257489060355
0.16442395060049955
0.13322596806881398


Estadísticas de Pyade
0.40136014609341314
0.7888948234514646
0.16505247772344778
0.017962089859151518
2.6383142469779868
Resultados de Kruskal en la función bohachevsky: KruskalResult(statistic=5.519750612335783, pvalue=0.06329966094484653)
Fitness de DE
0.0
8.225727693739369
6.303157303028167
9.987800262470241
4.166861933568242
6.681139237857854
18.95357636742272
5.127779329146744
17.292573993395564
0.0


Estadísticas de DE
7.67386161206289
6.3668645250286815
6.4921482704430105
0.0
18.95357636742272

Fitness de Scipy
8.325828324526784
8.094497616504462
6.375997645269012
8.198238434700265
7.933075120264752
7.994135129836874
6.641273525191726
9.095568071205532
7.632129435484974
8.476742131910347


Estadísticas de Scipy
7.876748543489473
0.8201218735467214
8.044316373170668
6.375997645269012
9.095568071205532

Fitness de Pyade
2.438692633612565
2.44726615164237
3.1354033355816053
2.322540198136492
2.5753713685192965
3.35240927654809
1.2028608291935952
2.925548439088487
2.7073126295018906
2.2761456933396196


Estadísticas de Pyade
2.538355055516401
0.5884150959800634
2.511318760080833
1.2028608291935952
3.35240927654809
Resultados de Kruskal en la función schaffer: KruskalResult(statistic=13.52817089452604, pvalue=0.0011545028653648453)
Resultados de Friedman: FriedmanchisquareResult(statistic=8.72727272727272, pvalue=0.012732005168250004)

Process finished with exit code 0
